In [13]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import BasicAer

from qiskit import *
import numpy as np
from qiskit.visualization import plot_bloch_multivector, plot_histogram
%matplotlib inline
import matplotlib.pyplot as plt

import time
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap, RealAmplitudes, EfficientSU2
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.optimizers import COBYLA

from qiskit.circuit import Parameter, ParameterVector
from qiskit.quantum_info import state_fidelity, random_statevector, hellinger_fidelity

from IPython.display import clear_output
from scipy.stats import entropy

## Solution

### Data loading 

In [3]:
data_path='./dataset'
data = np.loadtxt(data_path + "/foo.csv", 
                        delimiter=",")
print(data.shape)

(10000, 3)


### Pre-processing

Extract training, testing and validation sets from the data.

In [4]:
data_features = data[:, 1:]
data_labels = data[:, :1].reshape(data.shape[0],)

data_four = []
data_nine = []
for i in range(data.shape[0]):
    if data_labels[i] == 4:
        data_four.append(data_features[i])

for i in range(data.shape[0]):
    if data_labels[i] == 9:
        data_nine.append(data_features[i])
        
data_four = np.array(data_four)
data_nine = np.array(data_nine)

In [5]:
train_size = 20
test_size = 10
validation_size = 5

zero_train = data_four[:train_size]
one_train = data_nine[:train_size]

zero_test = data_four[train_size + 1:train_size + test_size + 1]
one_test = data_nine[train_size + 1:train_size + test_size + 1]

training_input = {'A':zero_train, 'B':one_train}
test_input = {'A':zero_test, 'B':one_test}

datapoints = []
dp_four = data_four[train_size + test_size + 2:train_size + test_size + 2 + validation_size]
dp_nine = data_nine[train_size + test_size + 2:train_size + test_size + 2 + validation_size]
datapoints.append(np.concatenate((dp_four, dp_nine)))
dp_y = np.array([0]*validation_size + [1]*validation_size)
datapoints.append(dp_y)
class_to_label = {'A': 4, 'B': 9}

In [6]:
print(datapoints)

[array([[0.49136811, 1.53639054],
       [1.08207047, 1.40790713],
       [0.85819936, 1.3985703 ],
       [0.83089256, 1.4313724 ],
       [0.96290463, 1.23597538],
       [0.99061573, 1.43646479],
       [1.18140972, 1.52991557],
       [0.55343014, 1.1865226 ],
       [0.38445497, 1.54558825],
       [0.74067843, 1.30667114]]), array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])]


### Defining our Parametric Quantum Circuit

In [16]:
def pqc_1(num_qubits=4, reps=1):
    num_qubits = num_qubits            
    reps = reps             # number of times you'd want to repeat the circuit

    num_params = int(reps*(num_qubits + 2*((num_qubits*(num_qubits - 1))/2)))
    x = ParameterVector('x', length=num_params)  # creating a list of Parameters
    variational_form = QuantumCircuit(num_qubits)

    num = -1
    # defining our parametric form
    for _ in range(reps):
        for i in range(num_qubits):
            num = num + 1
            variational_form.rx(x[num], i)

        for i in range(num_qubits):
            for j in range(i + 1, num_qubits):
                variational_form.cx(i, j)
                num = num + 1
                variational_form.u1(x[num], i)
                num = num + 1
                variational_form.u1(x[num], j)
                variational_form.cx(i, j)
    return variational_form

In [23]:
from qiskit.aqua.components.feature_maps import FeatureMap
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import BlueprintCircuit

class PQC1(FeatureMap):
    """Mapping data with a custom feature map."""
    
    def __init__(self, feature_dimension, depth=2, entangler_map=None):
        """
        Args:
            feature_dimension (int): number of features
            depth (int): the number of repeated circuits
            entangler_map (list[list]): describe the connectivity of qubits, each list describes
                                        [source, target], or None for full entanglement.
                                        Note that the order is the list is the order of
                                        applying the two-qubit gate.        
        """
        self._support_parameterized_circuit = False
        self._feature_dimension = feature_dimension
        self._num_qubits = self._feature_dimension = feature_dimension
        self._depth = depth
        self._entangler_map = None
        if self._entangler_map is None:
            self._entangler_map = [[i, j] for i in range(self._feature_dimension) for j in range(i + 1, self._feature_dimension)]
            
    def construct_circuit(self, x, qr, inverse=False):
        """Construct the feature map circuit.
        
        Args:
            x (numpy.ndarray): 1-D to-be-transformed data.
            qr (QauntumRegister): the QuantumRegister object for the circuit.
            inverse (bool): whether or not to invert the circuit.
            
        Returns:
            QuantumCircuit: a quantum circuit transforming data x.
        """
        qc = QuantumCircuit(qr)

        
#         for _ in range(self._depth):
#             for i in range(self._feature_dimension):
#                 qc.rx(x[i], qr[i])
#             for [source, target] in self._entangler_map:
#                 qc.cx(qr[source], qr[target])
#                 qc.u1(x[source] * x[target], qr[target])
#                 qc.cx(qr[source], qr[target])
                
#         num_params = int(reps*(self._num_qubits + 2*((self._num_qubits*(self._num_qubits - 1))/2)))
#         x = ParameterVector('x', length=num_params)  # creating a list of Parameters
#         variational_form = QuantumCircuit(self._num_qubits)

        num = -1
        # defining our parametric form
        for _ in range(self._depth):
            for i in range(self._num_qubits):
                num = num + 1
                qc.rx(x[num], i)

            for i in range(self._num_qubits):
                for j in range(i + 1, self._num_qubits):
                    qc.cx(i, j)
                    num = num + 1
                    qc.u1(x[num], i)
                    num = num + 1
                    qc.u1(x[num], j)
                    qc.cx(i, j)
                    
        if inverse:
            qc.inverse()
        return qc

In [26]:
def feature_map(): 
    # BUILD FEATURE MAP HERE
    feature_dim = 2
#     feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=2, insert_barriers=True)
#     feature_map = pqc_1(num_qubits=feature_dim, reps=2)
    feature_map = PQC1(feature_dimension=2, depth=2)
#     feature_map.draw()
    # BUILD FEATURE MAP HERE
    return feature_map

In [18]:
def variational_circuit():
    # BUILD VARIATIONAL CIRCUIT HERE
    num_qubits = 2
    var_circuit = EfficientSU2(num_qubits, entanglement='full', reps=2)
    var_circuit.draw()
    # BUILD VARIATIONAL CIRCUIT HERE
    return var_circuit

In [19]:
def classical_optimizer():
    # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT
    cls_opt = COBYLA(maxiter=500, tol=0.001)
    # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT
    return cls_opt

In [20]:
def call_back_vqc(eval_count, var_params, eval_val, index):
    print("eval_count: {}".format(eval_count))
    print("var_params: {}".format(var_params))
    print("eval_val: {}".format(eval_val))
    print("index: {}".format(index))

In [27]:
seed = 10598
backend = BasicAer.get_backend('qasm_simulator')
backend_options = {"method": "statevector_gpu"}
# backend = BasicAer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed, 
                                   backend_options=backend_options)

vqc = VQC(optimizer=classical_optimizer(), 
          feature_map=feature_map(), 
          var_form=variational_circuit(), 
          callback=call_back_vqc, 
          training_dataset=training_input, 
          test_dataset=test_input, 
          datapoints=datapoints[0])

/Users/ranaprathap/Library/Python/3.8/lib/python/site-packages/qiskit/aqua/algorithms/classifiers/vqc.py:146: DeprecationWarning: The qiskit.aqua.components.feature_maps.FeatureMap object is deprecated as of 0.7.0 and will be removed no earlier than 3 months after the release. You should pass a QuantumCircuit object instead. See also qiskit.circuit.library.data_preparation for a collection of suitable circuits.
  self.feature_map = feature_map


In [28]:
start = time.process_time()

result = vqc.run(quantum_instance)

print("time taken: ")
print(time.process_time() - start)


print("testing success ratio: {}".format(result['testing_accuracy']))

IndexError: index 2 is out of bounds for axis 0 with size 2

In [55]:
print(result)

{'num_optimizer_evals': 147, 'min_val': 0.488225885005347, 'opt_params': array([ 0.95555626,  1.31699211,  1.10265822, -0.58407942,  0.21431189,
        2.25005968,  0.49131123,  0.41872244,  1.35400569,  0.21243134,
       -0.34673529, -1.99825985]), 'eval_time': 556.5122737884521, 'eval_count': 147, 'training_loss': 0.488225885005347, 'testing_accuracy': 0.55, 'test_success_ratio': 0.55, 'testing_loss': 0.8032443730104154, 'predicted_probs': array([[0.51464844, 0.48535156],
       [0.54296875, 0.45703125],
       [0.48046875, 0.51953125],
       [0.43261719, 0.56738281],
       [0.63183594, 0.36816406],
       [0.4609375 , 0.5390625 ],
       [0.39355469, 0.60644531],
       [0.88574219, 0.11425781],
       [0.62695312, 0.37304688],
       [0.62792969, 0.37207031]]), 'predicted_labels': array([0, 0, 1, 1, 0, 1, 1, 0, 0, 0]), 'predicted_classes': ['A', 'A', 'B', 'B', 'A', 'B', 'B', 'A', 'A', 'A']}


In [66]:
datapoints[1]

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

In [69]:
accuracy = np.sum(np.array(datapoints[1]) == result['predicted_labels'])/len(ans[1])
print("Accuracy when datapoints are used for validation {}".format(accuracy))

Accuracy when datapoints are used for validation 0.5


In [56]:
optimal_params = np.array([ 0.95555626,  1.31699211,  1.10265822, -0.58407942,  0.21431189,
        2.25005968,  0.49131123,  0.41872244,  1.35400569,  0.21243134,
       -0.34673529, -1.99825985])

## Grading Function:

In [59]:
def grade(data, labels, feature_map, var_form, optimal_params):
    from qiskit.aqua.components.optimizers import COBYLA
    cobyla = COBYLA()
    backend = BasicAer.get_backend('qasm_simulator')
    backend_options = {"method": "statevector"}
    # backend = BasicAer.get_backend('statevector_simulator')
    quantum_instance = QuantumInstance(backend, shots=2000, seed_simulator=seed, seed_transpiler=seed, backend_options=backend_options)
    vqc = VQC(optimizer=cobyla, feature_map=feature_map, var_form=var_form, training_dataset=training_input)
    ans = vqc.predict(data, quantum_instance=quantum_instance, params=optimal_params)
    accuracy = np.sum(np.array(labels) == ans[1])/len(ans[1])
    return accuracy, ans

In [65]:
# let us use datapoints as our validation dataset
accuracy, ans = grade(data=datapoints[0], 
                      labels=datapoints[1], 
                      feature_map=feature_map(), 
                      var_form=variational_circuit(), 
                      optimal_params=optimal_params)
print("Accuracy of the model is {}".format(accuracy))

Accuracy of the model is 0.5


In [84]:
datapoints_l = []
temp = 1000
dp_four = data_four[train_size + test_size + 2:train_size + test_size + 2 + temp]
dp_nine = data_nine[train_size + test_size + 2:train_size + test_size + 2 + temp]
datapoints_l.append(np.concatenate((dp_four, dp_nine)))
dp_y = np.array([0]*temp + [1]*temp)
datapoints_l.append(dp_y)


In [85]:
datapoints_l

[array([[0.49136811, 1.53639054],
        [1.08207047, 1.40790713],
        [0.85819936, 1.3985703 ],
        ...,
        [0.87197   , 1.18651819],
        [0.49264258, 1.16549969],
        [0.69933271, 1.51421225]]),
 array([0, 0, 0, ..., 1, 1, 1])]

In [86]:
start = time.process_time()

accuracy, ans = grade(data=datapoints_l[0], 
                      labels=datapoints_l[1], 
                      feature_map=feature_map(), 
                      var_form=variational_circuit(), 
                      optimal_params=optimal_params)
print("Accuracy of the model is {}".format(accuracy))

print("time taken: ")
print(time.process_time() - start)


print("testing success ratio: {}".format(result['testing_accuracy']))

Accuracy of the model is 0.6335
time taken: 
31.324706000000006
testing success ratio: 0.55


In [87]:
a= 5

time.sleep(5)

print(a)

5
